In [2]:
!pip install keras

     ---------------------------------------- 1.7/1.7 MB 7.4 MB/s eta 0:00:00


In [4]:
!pip install tensorflow

     -------------------------------------- 272.8/272.8 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 6.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 5.6/5.6 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 5.7 MB/s eta 0:00:00
     ------------------------------------- 440.7/440.7 kB 13.9 MB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 5.5 MB/s eta 0:00:00
     ---------------------------------

In [ ]:
import keras
from keras.layers import Conv2D , MaxPooling2D, Flatten , Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model=Sequential()  ##help in going layer by layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [7]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [9]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [19]:
training_set=train_datagen.flow_from_directory(
             directory=r'C:\Users\shivansh.t\Desktop\opencv\train',
               target_size=(150,150),
               batch_size=16,
               class_mode='binary')

Found 4092 images belonging to 2 classes.


In [20]:
test_set=test_datagen.flow_from_directory(
             directory=r'C:\Users\shivansh.t\Desktop\opencv\test',
               target_size=(150,150),
               batch_size=16,
               class_mode='binary')

Found 716 images belonging to 2 classes.


In [21]:
history=model.fit_generator(
        training_set, 
        epochs=20,
        validation_data=test_set
)

C:\Users\shivansh.t\AppData\Local\Temp\ipykernel_15556\3791542443.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/20
112/256 [============>.................] - ETA: 1:04 - loss: 0.4356 - accuracy: 0.8075

C:\Users\shivansh.t\AppData\Local\anaconda3\lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


256/256 [==============================] - 130s 498ms/step - loss: 0.3330 - accuracy: 0.8617 - val_loss: 0.2415 - val_accuracy: 0.8883
Epoch 2/20
256/256 [==============================] - 127s 496ms/step - loss: 0.2285 - accuracy: 0.9157 - val_loss: 0.1923 - val_accuracy: 0.9092
Epoch 3/20
256/256 [==============================] - 125s 488ms/step - loss: 0.1890 - accuracy: 0.9260 - val_loss: 0.1176 - val_accuracy: 0.9581
Epoch 4/20
256/256 [==============================] - 127s 495ms/step - loss: 0.1795 - accuracy: 0.9291 - val_loss: 0.1121 - val_accuracy: 0.9637
Epoch 5/20
256/256 [==============================] - 127s 495ms/step - loss: 0.1603 - accuracy: 0.9409 - val_loss: 0.1184 - val_accuracy: 0.9567
Epoch 6/20
256/256 [==============================] - 127s 495ms/step - loss: 0.1511 - accuracy: 0.9416 - val_loss: 0.0778 - val_accuracy: 0.9679
Epoch 7/20
256/256 [==============================] - 126s 490ms/step - loss: 0.1399 - accuracy: 0.9460 - val_loss: 0.2930 - val_accura

In [22]:
model.save('mask_detector.h5',history)

In [1]:
import numpy as np
from keras.models import load_model
#from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
import cv2

In [2]:
mymodel=load_model('mask_detector.h5')

In [3]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+"Haarcascade_frontalface_default.xml")

In [16]:
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)  #Width of the frames in the video stream. 
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)  #height of the frames in the video stream.
while cap.isOpened():
    s,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)  ##detected faces in the frame
    
    for (x,y,w,h) in face:
        face_img=img[y:y+h,x:x+w]  #cropped the face dimension from the frame
        cv2.imwrite('temp.jpg',face_img)  #Saving that cropped image
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))  #loading that cropped image and targeting the dimension of face same as face which was trained upon
        test_image=image.img_to_array(test_image)  #converting image into array for calculation part
        test_image=np.expand_dims(test_image,axis=0)  #exapndaing dimension for clear detection 
        
        pred=mymodel.predict(test_image)[0][0]
        
        if pred==1:#pred==0   for NO mask 
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)  ##face detected rectange
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,0,255),-1)  ##rectangle in which text will be writen i.e "no mask"
            cv2.putText(img,"No Mask",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
            
        else:  ###pred==0   for mask on
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)  ##face detected rectange
            cv2.rectangle(img,(x,y-40),(x+w,y),(0,255,0),-1)##rectangle in which text will be writen i.e "mask"
            cv2.putText(img,"Mask",(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('Image',img)
    
    if cv2.waitKey(1) ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 25ms/step
